In [30]:
import pandas as pd
import numpy as np
import os

In [31]:
# path
path = r"C:\Users\USYS671257\OneDrive - WSP O365\21_31000110.002_Chattanooga TPO Model\model outputs"

# inputs
countycorr = pd.read_csv(os.path.join(path, "county_districts_chattanooga.csv"))
tripdata  = pd.read_csv(os.path.join(path, "_trip_2.dat"), sep = '\t')
hhdata  = pd.read_csv(os.path.join(path, "_household_2.dat"), sep = '\t')
perdata = pd.read_csv(os.path.join(path, "_person_2.dat"), sep = '\t')

# parameters
excludeChildren5 = True

In [32]:
def prep_perdata(perdata,hhdata):

    countycorr_dict = countycorr.set_index("TAZID")["District"].to_dict()
    hhdata["hhcounty"] = hhdata["hhtaz"].map(countycorr_dict)
    perdata = perdata.merge(hhdata, on="hhno", how="left")
    
    return perdata



def prep_tripdata(tripdata, perdata, hhdata):
    
    countycorr_dict = countycorr.set_index("TAZID")["District"].to_dict()
    
    perdata = prep_perdata(perdata, hhdata)
    perdata = perdata[["hhno","pno","pptyp","hhtaz","hhcounty","pwtaz","psexpfac"]]
    
    tripdata = pd.merge(tripdata, perdata, on=["hhno","pno"], how="left")
    if excludeChildren5:
        tripdata = tripdata[tripdata["pptyp"]<8]
    
    tripdata["ocounty"] = tripdata["otaz"].map(countycorr_dict)
    tripdata["dcounty"] = tripdata["dtaz"].map(countycorr_dict)
    tripdata["distcat"] = pd.cut(tripdata["travdist"], 
                                    bins=range(0, 91),  
                                    right=True,
                                    labels=list(range(0, 90)))
    tripdata["timecat"] = pd.cut(tripdata["travtime"], 
                                    bins=range(0, 91),  
                                    right=True,
                                    labels=list(range(0, 90)))
    tripdata["wrkrtyp"] = np.where(tripdata.pptyp==1, "FT", 
                                   np.where(tripdata.pptyp==2, "PT","NotFTPT"))
    tripdata["travtime"] = np.where(tripdata.travtime<0, np.NaN, tripdata.travtime)
    tripdata["travdist"] = np.where(tripdata.travdist<0, np.NaN, tripdata.travdist)
    
    return tripdata
    
    

def summary_trip_destination(dstripdata, purpose, sum_by_var):
    
    """ trip purpose
        none/home 0 
        Work 1 
        School 2 
        Escort 3
        Personal_Business 4
        Shop 5
        Meal 6
        SocRec 7
        recreational 8 
        medical 9
    """
       
    dstripdata = dstripdata[dstripdata["dpurp"]==purpose]
    index_order = list(range(0, 91))
    summary = (dstripdata.groupby([sum_by_var])["psexpfac"].
                         sum().
                         reindex(index=index_order).
                         fillna(0).
                         reset_index())
    return summary

In [33]:
# clean up person data
dstripdata = prep_tripdata(tripdata, perdata, hhdata)

In [34]:
# trip destination - Work 1 
summary_trip_destination(dstripdata, 1, "distcat")

,distcat,psexpfac
0,0,11366.0
1,1,19750.0
2,2,20081.0
3,3,20194.0
4,4,18284.0
...,...,...
86,86,0.0
87,87,0.0
88,88,0.0
89,89,0.0


In [35]:
# trip destination - Work 1 
summary_trip_destination(dstripdata, 1, "timecat")

,timecat,psexpfac
0,0,2939.0
1,1,6570.0
2,2,7343.0
3,3,8321.0
4,4,8466.0
...,...,...
86,86,45.0
87,87,37.0
88,88,45.0
89,89,38.0
